In [1]:
import jpeg4py as jpeg
from torch.utils.data import Dataset, DataLoader
import time
import pyspng

In [12]:
import time
from tqdm.notebook import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import DatasetFolder
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import os


import cv2
import jpeg4py as jpeg


import cv2
from PIL import Image


from functools import partial


def pil_loader(path):
    """
    Returns:
        PIL Image
    """
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
def opencv_loader(path):
    """
    Returns:
        images(np.array [H, W, C])
    """
    image = cv2.imread(path)
    return image

def jpeg4py_loader(path):
    """
    Returns:
        images(np.array [H, W, C])
    """
    image = jpeg.JPEG(path).decode()
    return image

def jp4pil_loader(path):
    """
    Returns:
        images(np.array [H, W, C])
    """
    image = jpeg4py_loader(path)
    return Image.fromarray(image.astype('uint8'), 'RGB')

# def tfms_deco(original_func, image):
#     return original_func(image)

class CustomDataset(Dataset):
    IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
    
    def __init__(self, root_path, transform=None, loader_type='pil'):
        self.root_path = root_path
        self.file_list = [f for f in os.listdir(root_path) if f.endswith('.jpg')][:1000]
        self.loader = self._make_loader(loader_type)
        self.loader_type = loader_type
        if transform is not None:
            self.transform = transform
        else:
            self._make_transforms()
        
    # def _setup_transform(self, transform):
    #     """
    #     albumentation은 사용할 때 지정된 signature 그대로 사용해야 함
    #     """
    #     if transform.__class__ == 'albumentations.core.composition.Compose':
    #         def albm_tfms(image):
    #             return transform(image=image)['image']
    #         self.transform = albm_tfms
    #     else:
    #         self.transform = transform

    def _make_transforms(self):
        if self.loader_type == 'pil':
            self.transform = transforms.Compose([transforms.ToTensor()])
        elif self.loader_type == 'opencv':
            def opencv_tfms(img):
                return torch.from_numpy(img.transpose(2, 0, 1))
            self.transform = opencv_tfms
        elif self.loader_type == 'jpeg4py':
            def jpeg4py_tfms(img):
                return torch.from_numpy(img.transpose(2, 0, 1))
            self.transform = jpeg4py_tfms
        else:
            self.transform = None

            
    def _make_loader(self, loader_type):
        if loader_type == 'opencv':
            return opencv_loader
        elif loader_type == 'jpeg4py':
            return jpeg4py_loader
        elif loader_type =='jp4pil':
            return jp4pil_loader
        else:
            return pil_loader

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path= self.file_list[idx]
        # File Loading
        img = self.loader(os.path.join(self.root_path, img_path))
        if self.transform is not None:
            img = self.transform(img)
        return img

In [13]:
root_path = '/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/training/images'

In [14]:
custom_ds = CustomDataset(root_path, loader_type='pil')

In [15]:
custom_ds.file_list[:10]

['MEN-Tees_Tanks-id_00003425-05_3_back.jpg',
 'WOMEN-Tees_Tanks-id_00000871-02_2_side.jpg',
 'WOMEN-Dresses-id_00002631-05_7_additional.jpg',
 'WOMEN-Tees_Tanks-id_00005310-06_4_full.jpg',
 'WOMEN-Leggings-id_00000286-04_1_front.jpg',
 'WOMEN-Tees_Tanks-id_00002136-02_1_front.jpg',
 'WOMEN-Blouses_Shirts-id_00006642-04_2_side.jpg',
 'WOMEN-Sweaters-id_00001635-06_7_additional.jpg',
 'WOMEN-Cardigans-id_00001436-02_7_additional.jpg',
 'WOMEN-Graphic_Tees-id_00006415-01_4_full.jpg']

In [22]:
custom_ds[2].shape

torch.Size([3, 1101, 750])

In [23]:
dataloader = DataLoader(custom_ds, batch_size=16, shuffle=False, num_workers=0)

In [ ]:
# 사이즈가 일관되지 않아서 Resize해야 함
%%timeit -r 3 -n 3
simple_load_times = []
start_time = time.time()
for image in dataloader:
    image = image.cuda()
    pass
pil_time = time.time() - start_time
simple_load_times.append(pil_time)
print(str(simple_load_times) + ' sec')

In [27]:
import cv2
from einops import rearrange, repeat

source = cv2.imread('/home/aiteam/tykim/generative_model/framework/diffusion/ControlNet/training/images/MEN-Denim-id_00000089-02_7_additional.jpg')
source = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
source = source.astype(np.float32) / 255.0
source.shape

(1101, 750, 3)

In [31]:
source = source[None, ...]

In [32]:
x = rearrange(source, 'b h w c -> b c h w')

In [33]:
type(x)

numpy.ndarray

In [34]:
x=x.to(memory_format=torch.contiguous_format).float()

AttributeError: 'numpy.ndarray' object has no attribute 'to'